日->英 翻訳器をつくります．

まずは必要なデータやライブラリをそろえます．

In [ ]:
! git clone https://github.com/nymwa/light_enja2.git
! mv light_enja2/* .
! pip install -r requirements.txt

Cloning into 'light_enja2'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 30 (delta 4), reused 30 (delta 4), pack-reused 0
Unpacking objects: 100% (30/30), done.
     |████████████████████████████████| 860kB 30.2MB/s 
     |████████████████████████████████| 1.7MB 46.5MB/s 
     |████████████████████████████████| 61kB 10.1MB/s 
     |████████████████████████████████| 1.2MB 38.4MB/s 
     |████████████████████████████████| 153kB 49.1MB/s 
     |████████████████████████████████| 112kB 57.5MB/s 
     |████████████████████████████████| 81kB 9.6MB/s 
     |████████████████████████████████| 645kB 49.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp37-none-any.whl size=883990 sha256=1572d7087702ed4771c6b586f584c0cb4056815914ba58f1b8b2369dde9871bb
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
  Created wheel for a

データに対して前処理を行います

ダウンロードした時点でデータセットは単語分割やトークン化がすでに施されています．
マージ数8000のBPEでサブワードに分割します．

In [ ]:
! bash preproc.sh

+ TRAIN_EN=corpus/train.en
+ TRAIN_JA=corpus/train.ja
+ VALID_EN=corpus/valid.en
+ VALID_JA=corpus/valid.ja
+ TEST_EN=corpus/test.en
+ cat corpus/train.en corpus/train.ja
+ python src/learn.py --input train.enja --prefix bpe --vocab-size 4000 --character-coverage 0.9995 --threads 1
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: train.enja
  input_format: 
  model_prefix: bpe
  model_type: BPE
  vocab_size: 4000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 1
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_voc

学習を行います．

少ないデータでかつかなり軽量なモデルでの学習ですが，10分ぐらいかかります．

In [ ]:
!fairseq-train \
    data-bin \
    --save-interval 10 \
    --max-epoch 10 \
    --update-freq 1 \
    --max-tokens 4000 \
    --arch transformer \
    --encoder-normalize-before \
    --decoder-normalize-before \
    --encoder-embed-dim 512 \
    --encoder-ffn-embed-dim 1024 \
    --encoder-attention-heads 4 \
    --encoder-layers 4 \
    --decoder-embed-dim 512 \
    --decoder-ffn-embed-dim 1024 \
    --decoder-attention-heads 4 \
    --decoder-layers 4 \
    --share-all-embeddings \
    --dropout 0.3 \
    --optimizer adam \
    --adam-betas '(0.9, 0.999)' \
    --lr 0.002 \
    --lr-scheduler inverse_sqrt \
    --warmup-updates 2000 \
    --warmup-init-lr 1e-07 \
    --clip-norm 1.0 \
    --weight-decay 0.01 \
    --criterion label_smoothed_cross_entropy \
    --label-smoothing 0.6

2021-07-17 12:50:41 | INFO | fairseq_cli.train | Namespace(activation_dropout=0.0, activation_fn='relu', adam_betas='(0.9, 0.999)', adam_eps=1e-08, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, all_gather_list_size=16384, arch='transformer', attention_dropout=0.0, batch_size=None, batch_size_valid=None, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=1.0, cpu=False, criterion='label_smoothed_cross_entropy', cross_self_attention=False, curriculum=0, data='data-bin', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_attention_heads=4, decoder_embed_dim=512, decoder_embed_path=None, decoder_ffn_embed_dim=1024, decoder_input_dim=512, decoder_layerdrop=0, decoder_layers=4, decoder_layers_to_keep=None, decoder_learned_pos=False, decoder_normalize_before=True, decoder_output_dim=512, device_id=0, disable_validation=False, distributed_bac

テストデータで翻訳を行い，その性能を評価します．

評価尺度BLEUで30ぐらい出ると思います．

In [ ]:
! fairseq-interactive data-bin \
    --buffer-size 1024 \
    --batch-size 128 \
    --path checkpoints/checkpoint10.pt \
    --beam 5 \
    --lenpen 0.6 \
    < test.en \
    | grep '^H' \
    | cut -f 3 \
    | python src/decode.py \
    | tee output.txt \
    | sacrebleu corpus/test.ja

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
BLEU+case.mixed+numrefs.1+smooth.exp+tok.13a+version.1.5.1 = 30.6 66.3/41.9/27.1/18.6 (BP = 0.891 ratio = 0.896 hyp_len = 45908 ref_len = 51230)


翻訳結果と正解データの冒頭10文です．

In [ ]:
! head output.txt corpus/test.ja

==> output.txt <==
彼女 の 母 は 美しい 女性 です 。
畑 に は 6 人 の 羊 が い た 。
今朝 は 散歩 の 間 に 大きな 写真 を 撮る こと が でき た 。
今晩 電話 し ます 。
彼女 は 本 の 虫 だ 。
式 は 終わっ た 。
私 は 彼 に 前日 自分 で 楽しん だ か どう か 尋ね た 。
彼 は 注意 し て い ない 。
列車 は いつも 定刻 に いる 。
2 時 に 出発 すれ ば 、 彼 ら は 6 時 に 着く べき だ 。

==> corpus/test.ja <==
彼女 の お母さん は きれい な 女 の 人 です 。
野原 に は 六 頭 の 羊 が い た 。
この 朝 の 散歩 で とても 素晴らしい 写真 を 撮る こと が でき た 。
今夜 電話 し ます 。
彼女 は いわゆる 本 の 虫 です 。
式典 は 終わり まし た 。
昨日 は 楽しかっ た か 、 と 私 は 彼 に 尋ね た 。
彼 に は 心配 事 が ない 。
その 列車 は いつも 時刻 通り だ 。
2 時 に 出発 し たら 、 6 時 に は つく はず だ 。


たのしい

In [ ]:
import re
import unicodedata
from sacremoses import MosesTokenizer
import sentencepiece as spm
from fairseq.models.transformer import TransformerModel

In [ ]:
mt = MosesTokenizer(lang = 'en')
sp = spm.SentencePieceProcessor(model_file='bpe.model')
model = TransformerModel.from_pretrained('./checkpoints/', checkpoint_file='checkpoint10.pt', data_name_or_path='data-bin')

def preproc_en(x):
  x = unicodedata.normalize('NFKC', x)
  x = re.sub(mt.AGGRESSIVE_HYPHEN_SPLIT[0], r'\1 - ', x)
  x = mt.tokenize(x, escape = False)
  x = ' '.join(x)
  x = x.lower()
  x = ' '.join(sp.encode(x, out_type = 'str'))
  return x

def translate(x):
  x = preproc_en(x)
  x = model.translate(x, beam = 5, lenpen = 0.6)
  x = ''.join(x.split()).replace('▁', '').strip()
  return x

In [ ]:
while True:
  x = input('英文を入力 > ')
  if not x:
    break
  x = translate(x)
  print('翻訳結果 > ' + x)

英文を入力 > Why is the baby crying?


/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


翻訳結果 > 赤ん坊は何で泣いているの?
英文を入力 > There was no one left but me.
翻訳結果 > 私には誰も残っていなかった。
英文を入力 > I know how old you are.
翻訳結果 > あなたが何歳か知っている。
英文を入力 > 
